In [1]:
from typing import Dict

from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.llms.azureml_endpoint import AzureMLOnlineEndpoint, DollyContentFormatter
import json
import os

examples = [
    {"word": "treasure", "antonym": "trash"},
    {"word": "loud", "antonym": "quiet"},
    {"word": "angry", "antonym": "calm"},
]
formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=formatter_template
)

few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every word\n",
    suffix="Word: {input}\nAntonym: ",
    input_variables=["input"],
    example_separator="\n",
)

content_formatter = DollyContentFormatter()

llm = AzureMLOnlineEndpoint(
    endpoint_api_key=os.getenv("ENDPOINT_API_KEY"),
    endpoint_url=os.getenv("ENDPOINT_URL"),
    deployment_name="databricks-dolly-v2-12b-4",
    content_formatter=content_formatter
)

resp = llm(few_shot_template.format(input="soft"))
resp

The request failed with status code: 401


JSONDecodeError: Expecting value: line 1 column 1 (char 0)